## Debug RFT

In [115]:
import torch
import math

num_envs = 1
num_bodies = 2
num_contact_points = 10*10

contact_edge_x = (-0.06, 0.14)
contact_edge_y = (-0.03, 0.03)
surface_area = (contact_edge_x[1]-contact_edge_x[0])*(contact_edge_y[1]-contact_edge_y[0])

In [116]:
def rpy_to_matrix(rpy: torch.Tensor) -> torch.Tensor:
    """
    Convert batched (roll, pitch, yaw) to rotation matrices.

    Args:
        rpy: (B, 3) tensor in radians as (roll, pitch, yaw) = (Rx, Ry, Rz)
    Returns:
        R: (B, 3, 3) rotation matrices
    Convention:
        R = Rz(yaw) @ Ry(pitch) @ Rx(roll)   # body->world for column vectors
    """
    assert rpy.ndim == 2 and rpy.size(-1) == 3, "Expected shape (B,3)"
    r, p, y = rpy.unbind(-1)

    cr, sr = torch.cos(r), torch.sin(r)
    cp, sp = torch.cos(p), torch.sin(p)
    cy, sy = torch.cos(y), torch.sin(y)
    
    R = torch.zeros((rpy.shape[0], 3, 3), dtype=rpy.dtype, device=rpy.device)

    # First row
    R[..., 0, 0] = cy * cp
    R[..., 0, 1] = cy * sp * sr - sy * cr
    R[..., 0, 2] = cy * sp * cr + sy * sr
    # Second row
    R[..., 1, 0] = sy * cp
    R[..., 1, 1] = sy * sp * sr + cy * cr
    R[..., 1, 2] = sy * sp * cr - cy * sr
    # Third row
    R[..., 2, 0] = -sp
    R[..., 2, 1] = cp * sr
    R[..., 2, 2] = cp * cr

    return R.to(torch.float32)


In [147]:
from typing import Literal
def _axis_angle_rotation(axis: Literal["X", "Y", "Z"], angle: torch.Tensor) -> torch.Tensor:
    """Return the rotation matrices for one of the rotations about an axis of which Euler angles describe,
    for each value of the angle given.

    Args:
        axis: Axis label "X" or "Y or "Z".
        angle: Euler angles in radians of any shape.

    Returns:
        Rotation matrices. Shape is (..., 3, 3).

    Reference:
        https://github.com/facebookresearch/pytorch3d/blob/main/pytorch3d/transforms/rotation_conversions.py#L164-L191
    """
    cos = torch.cos(angle)
    sin = torch.sin(angle)
    one = torch.ones_like(angle)
    zero = torch.zeros_like(angle)

    if axis == "X":
        R_flat = (one, zero, zero, zero, cos, -sin, zero, sin, cos)
    elif axis == "Y":
        R_flat = (cos, zero, sin, zero, one, zero, -sin, zero, cos)
    elif axis == "Z":
        R_flat = (cos, -sin, zero, sin, cos, zero, zero, zero, one)
    else:
        raise ValueError("letter must be either X, Y or Z.")

    return torch.stack(R_flat, -1).reshape(angle.shape + (3, 3))

def matrix_from_euler(euler_angles: torch.Tensor, convention: str) -> torch.Tensor:
    """
    Convert rotations given as Euler angles (intrinsic) in radians to rotation matrices.

    Args:
        euler_angles: Euler angles in radians. Shape is (..., 3).
        convention: Convention string of three uppercase letters from {"X", "Y", and "Z"}.
            For example, "XYZ" means that the rotations should be applied first about x,
            then y, then z.

    Returns:
        Rotation matrices. Shape is (..., 3, 3).

    Reference:
        https://github.com/facebookresearch/pytorch3d/blob/main/pytorch3d/transforms/rotation_conversions.py#L194-L220
    """
    if euler_angles.dim() == 0 or euler_angles.shape[-1] != 3:
        raise ValueError("Invalid input euler angles.")
    if len(convention) != 3:
        raise ValueError("Convention must have 3 letters.")
    if convention[1] in (convention[0], convention[2]):
        raise ValueError(f"Invalid convention {convention}.")
    for letter in convention:
        if letter not in ("X", "Y", "Z"):
            raise ValueError(f"Invalid letter {letter} in convention string.")
    matrices = [_axis_angle_rotation(c, e) for c, e in zip(convention, torch.unbind(euler_angles, -1))]
    # return functools.reduce(torch.matmul, matrices)
    return torch.matmul(torch.matmul(matrices[0], matrices[1]), matrices[2])

In [148]:
num_contact_point_side = int(math.sqrt(num_contact_points))
contact_point_offset_x = torch.linspace(contact_edge_x[0], contact_edge_x[1], num_contact_point_side)
contact_point_offset_y = torch.linspace(contact_edge_y[0], contact_edge_y[1], num_contact_point_side)
contact_point_offset_y, contact_point_offset_x = torch.meshgrid(contact_point_offset_y, contact_point_offset_x, indexing='ij')
contact_point_offset = torch.stack((
    contact_point_offset_x.flatten(), 
    contact_point_offset_y.flatten(), 
    -0.035 * torch.ones_like(contact_point_offset_x).flatten()
    ), dim=-1)
contact_point_offset = contact_point_offset.unsqueeze(0).unsqueeze(1).repeat(num_envs, num_bodies, 1, 1)

In [298]:
def rot_x(angle):
    """
    angle: (batch_size, )
    """
    c = torch.cos(angle)
    s = torch.sin(angle)
    R = torch.zeros((angle.shape[0], 3, 3), device=angle.device)
    R[:, 0, 0] = 1
    R[:, 1, 1] = c
    R[:, 1, 2] = -s
    R[:, 2, 1] = s
    R[:, 2, 2] = c
    return R

def rot_y(angle):
    """
    angle: (batch_size,)
    """
    c = torch.cos(angle)
    s = torch.sin(angle)
    R = torch.zeros((angle.shape[0], 3, 3), device=angle.device)
    R[:, 0, 0] = c
    R[:, 0, 2] = s
    R[:, 1, 1] = 1
    R[:, 2, 0] = -s
    R[:, 2, 2] = c
    return R

def rot_z(angle):
    """
    angle: (batch_size, )
    """
    c = torch.cos(angle)
    s = torch.sin(angle)
    R = torch.zeros((angle.shape[0], 3, 3), device=angle.device)
    R[:, 0, 0] = c
    R[:, 0, 1] = -s
    R[:, 1, 0] = s
    R[:, 1, 1] = c
    R[:, 2, 2] = 1
    return R

def matrix_from_euler_xyz(euler_angles: torch.Tensor) -> torch.Tensor:
    """
    Convert rotations given as Euler angles (intrinsic) in radians to rotation matrices.

    Args:
        euler_angles: Euler angles in radians. Shape is (..., 3).
    Returns:
        Rotation matrices. Shape is (..., 3, 3).    
        
    """
    
    if euler_angles.dim() == 0 or euler_angles.shape[-1] != 3:
        raise ValueError("Invalid input euler angles.")
    matrices = [rot_x(euler_angles[:, 0]), rot_y(euler_angles[:, 1]), rot_z(euler_angles[:, 2])]
    return matrices[2] @ matrices[1] @ matrices[0]

In [337]:
body_pos = torch.zeros((num_envs, num_bodies, 3))
body_pos[:, 0, 0] = 0.05
body_pos[:, 1, 0] = 0.05
body_pos[:, 0, 1] = 0.05
body_pos[:, 1, 1] = -0.05
body_pos[:, 0, 2] = 0.0
body_pos[:, 1, 2] = 0.0

v_body = torch.tensor([1.0, 0.0, -1.0])
w_body = torch.tensor([0.0, 0.5, 0.0])


euler_angle = torch.tensor([0, 0, math.pi/2]).unsqueeze(0).unsqueeze(1).repeat(num_envs, num_bodies, 1)
euler_angle[:, 0, 0] = -math.pi/18
euler_angle[:, 1, 0] = math.pi/18
body_mat = matrix_from_euler(euler_angle.view(-1, 3), "XYZ").view(num_envs, num_bodies, 3, 3)
body_lin_vel = (body_mat.reshape(-1, 3, 3) @ v_body.unsqueeze(0).unsqueeze(2)).unsqueeze(2).reshape(num_envs, num_bodies, 3)
body_ang_vel = (body_mat.reshape(-1, 3, 3) @ w_body.unsqueeze(0).unsqueeze(2)).unsqueeze(2).reshape(num_envs, num_bodies, 3)

In [338]:
print(body_lin_vel)
print(body_ang_vel)

tensor([[[-4.3711e-08,  8.1116e-01, -1.1585e+00],
         [-4.3711e-08,  1.1585e+00, -8.1116e-01]]])
tensor([[[-5.0000e-01, -2.1524e-08,  3.7952e-09],
         [-5.0000e-01, -2.1524e-08, -3.7952e-09]]])


In [339]:
contact_point_pos = body_pos.unsqueeze(2) + torch.einsum('ebij,ebpj->ebpi', body_mat, contact_point_offset)
contact_point_lin_vel = body_lin_vel.unsqueeze(2) + torch.cross(
    body_ang_vel.unsqueeze(2), contact_point_pos - body_pos.unsqueeze(2),
    dim=-1
)
# contact_point_lin_vel_b = torch.einsum('ebji,ebpj->ebpi', body_mat.transpose(-1, -2), contact_point_lin_vel.clone())
# contact_point_lin_vel_b = (body_mat.reshape(-1, 3, 3) @ v_body.unsqueeze(0).unsqueeze(2)).unsqueeze(2).reshape(num_envs, num_bodies, 3)
R_T = body_mat.transpose(-1, -2).unsqueeze(2)      # (E, B, 1, 3, 3)
v    = contact_point_lin_vel.unsqueeze(-1)         # (E, B, P, 3, 1)
contact_point_lin_vel_b = (R_T @ v).squeeze(-1)    # (E, B, P, 3)

contact_point_intrusion_angle = torch.atan2(-contact_point_lin_vel_b[..., 2], contact_point_lin_vel_b[..., 0])
contact_point_intrusion_angle = torch.where(
    contact_point_intrusion_angle > torch.pi/2, 
    torch.pi - contact_point_intrusion_angle, 
    contact_point_intrusion_angle
)
contact_point_intrusion_angle = torch.where(
    contact_point_intrusion_angle < -torch.pi/2, 
    -torch.pi - contact_point_intrusion_angle, 
    contact_point_intrusion_angle
)

In [336]:
contact_point_intrusion_angle * 180/math.pi

tensor([[[44.6332, 44.9595, 45.2821, 45.6011, 45.9165, 46.2283, 46.5367,
          46.8415, 47.1430, 47.4411, 44.6332, 44.9595, 45.2821, 45.6011,
          45.9165, 46.2283, 46.5367, 46.8415, 47.1430, 47.4411, 44.6332,
          44.9595, 45.2821, 45.6011, 45.9165, 46.2283, 46.5367, 46.8415,
          47.1430, 47.4411, 44.6332, 44.9595, 45.2821, 45.6011, 45.9165,
          46.2283, 46.5367, 46.8415, 47.1430, 47.4411, 44.6332, 44.9595,
          45.2821, 45.6011, 45.9165, 46.2283, 46.5367, 46.8415, 47.1430,
          47.4411, 44.6332, 44.9595, 45.2821, 45.6011, 45.9165, 46.2283,
          46.5367, 46.8415, 47.1430, 47.4411, 44.6332, 44.9595, 45.2821,
          45.6011, 45.9165, 46.2283, 46.5367, 46.8415, 47.1430, 47.4411,
          44.6332, 44.9595, 45.2821, 45.6011, 45.9165, 46.2283, 46.5367,
          46.8415, 47.1430, 47.4411, 44.6332, 44.9595, 45.2821, 45.6011,
          45.9165, 46.2283, 46.5367, 46.8415, 47.1430, 47.4411, 44.6332,
          44.9595, 45.2821, 45.6011, 45.9165, 46.22